# Librerie

In [ ]:
# installo le librerie
!pip install pydicom
!pip install pandas>=1.2.0
!pip install xlrd>=1.2.0.
!pip install --upgrade xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 96 kB 4.1 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
# importo le librerie
import sys  # system for info
import matplotlib.pyplot as plt
import numpy as np  # numpy
from PIL import Image
import pandas as pd # pandas
from pathlib import Path    #path library for paths management
from tqdm import tqdm   #tqdm for loading bars
import os   # os for paths and dirs
import pydicom  # pydicom library for dicom file import
from tensorflow import keras # per caricare il modello

# UTILS

## UTILS PATHS

In [ ]:
# crea i path necessari
databasePathIMAGES = Path('drive/MyDrive/Dataset/TrainSet (1)/TrainSet/')     #path of the images png

## RANDOM NUMBER

In [ ]:
random_seed = 42    #random seed per poter riprodurre l'esperimento

## GLOBAL VARIABLES

In [ ]:
IMAGE_SIZE = 224  #dimensione delle immagini in input (224x224)
CLASS_NAMES = ['MILD', 'SEVERE']    #valori delle label T

# FUNCTIONS

### Loader Dicom from Winner Hackathon

In [ ]:
def clahe_transform(img):
    # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(4, 4))
    img = clahe.apply((img * 255).astype(np.uint8)) / 255
    return img

def normalize(img, min_val=None, max_val=None):
    if not min_val:
        min_val = img.min()
    if not max_val:
        max_val = img.max()
    img = (img - min_val) / (max_val - min_val)
    # img -= img.mean()
    # img /= img.std()
    return img

def load_img(img_path):
    filename, extension = os.path.splitext(img_path)
    if extension == ".dcm":
        dicom = pydicom.dcmread(img_path)
        img = dicom.pixel_array.astype(float)
        photometric_interpretation = dicom.PhotometricInterpretation
    else:
        img = Image.open(img_path)
        img = np.array(img).astype(float)
        photometric_interpretation = 'MONOCHROME1'
    return img, photometric_interpretation


# def loader(img_path, img_dim, mask_path=None, box=None, clahe=False):
def loader(img_path, img_dim, clahe=False):
    # Img
    # img_vec = []
    img, photometric_interpretation = load_img(img_path)
    min_val, max_val = img.min(), img.max()

    # Pathometric Interpretation
    if photometric_interpretation == 'MONOCHROME1':
        img = np.interp(img, (min_val, max_val), (max_val, min_val))

    # To Grayscale
    # if img.ndim > 2:
        # img = img.mean(axis=2)

    # plt.imshow(img)

    # Resize
    img = cv2.resize(img, (img_dim, img_dim))
    # Normalize
    img = normalize(img, min_val=min_val, max_val=max_val)
    # CLAHE
    if clahe:
        img = clahe_transform(img)
    
    # To 3 Channels
    img = np.stack((img, img, img), axis=-1)
    
    return img

def loadAllImgToArray(DB,dimention,cName,basePt):
    paths = DB[cName].to_list()

    # print(paths)
    imgs = []
    for p in tqdm(paths):
        img = loader(basePt / p , dimention, clahe= True)
        # for i in img:
        imgs.append(img)


    imgs = np.array(imgs)
    return imgs
    

# Load From DB

### Import Database DICOM

In [ ]:
# monto il drive dal quale prendere le immagini
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Model load and classification

## Resnet50

In [ ]:
# carico il modello pretrainato
model = keras.models.load_model('drive/MyDrive/my_model.h5')

# GRAD-CAM++

In [ ]:
# installo le librerie per il framework gradcam
!pip install utils
!pip install pytorch-gradcam
!git clone https://github.com/samson6460/tf_keras_gradcamplusplus.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.0 MB 8.0 MB/s 
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-py3-none-any.whl size=5269 sha256=9dfb84dd7e894c3c3e5d9ecec35e95aee01c629fa1fe702d20e9b47265a184dd
  Stored in directory: /root/.cache/pip/wheels/06/e7/da/b13a71980c3a787414e5ff8e156701c561c6322636396d71cc
Successfully built pytorch-gradcam
Cloning into 'tf_keras_gradcamplusplus'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 100 (delta 35), reused 80 (delta 21), pack-reused 0
Receiving objects: 100% (100/100), 13.66 MiB | 25.25 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
# importo le librerie per il framework gradcam
sys.path.append('tf_keras_gradcamplusplus')
import utils #Dempster-Shafer layer
import gradcam #Utility layer for training
from tf_keras_gradcamplusplus.utils import vgg16_mura_model, preprocess_image, show_imgwithheat
from tf_keras_gradcamplusplus.gradcam import grad_cam, grad_cam_plus
import math
import cv2

In [ ]:
# imposto le classi
class_names = ['MILD', 'SEVERE']

In [ ]:
# funzione che preso in input il nome di un'immagine, fornisce in output la label assegnata e l'heatmap relativa
def single_img(img_name):
  s = {'ImageFile':[img_name]}
  image_df = pd.DataFrame(s)
  loaded_image = loadAllImgToArray(image_df,IMAGE_SIZE,'ImageFile',databasePathIMAGES)

  heatmap_plus = grad_cam_plus(
    model,
    loaded_image[0],
    #layer_name = "conv5_block3_out",
     layer_name = "conv4_block6_out",
    # layer_name = "conv3_block4_out",
    # layer_name = "conv2_block3_out",
    label_name = ['MILD', 'SEVERE']
    )
  
  show_imgwithheat('drive/MyDrive/Dataset/TrainSet (1)/TrainSet/'+image_df.at[0, 'ImageFile'], heatmap_plus, alpha = 0.3)

single_img('P_2_50.png')

In [ ]:
# funzione in ausilio a multi_img che, passate una heatmap e un immagine sovrappone le due
def visualizeGradCAMplus(img,hm, alpha = 0.4,outputArray = True):
    heatmap = cv2.resize(hm, (img.shape[1], img.shape[0]))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    imgwithheat = Image.fromarray(superimposed_img)
    return imgwithheat if outputArray else plt.imshow(imgwithheat)

In [ ]:
# funzione che preso in input più nomi di immagini, fornisce in output le relative label assegnate con le heatmap associate
def multi_img(img_name):

  s = {'ImageFile':img_name}
  image_df = pd.DataFrame(s)

  # print(image_df.type())

  loaded_image = loadAllImgToArray(image_df,IMAGE_SIZE,'ImageFile',databasePathIMAGES)

  edge = math.ceil(math.sqrt(64))
  i = 0

  figure = plt.figure()
  figure.set_size_inches(50, 30)

  for i in range(len(image_df)):
      im = loaded_image[i]

      if i>63:
          break
      ax = plt.subplot(edge, edge, i + 1)

      # hm = gradcam.grad_cam(CNN,im,layer_name='average_pooling2d_4_CNN')
      hm = grad_cam_plus(model,im,layer_name='conv5_block3_out', label_name = ['MILD', 'SEVERE'])

      img_hm = visualizeGradCAMplus(im,hm)

      plt.imshow(img_hm,)

      im2pred = np.reshape(im,(1,224,224,3))
      lab = class_names[0]
      pr = round(model.predict(im2pred)[0][0])
      pr = class_names[int(pr)]
      
      plt.axis("off")

      plt.axis("off")
      
      i = i+1

multi_img(['P_2_50.png', 'P_2_51.png', 'P_2_52.png', 'P_2_53.png', 'P_2_55.png'])